<a href="https://colab.research.google.com/github/minwoongPark/TEST/blob/main/ClassifyMnist1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MNIST image 분류 [출처](https://github.com/wikibook/dl-vision)
> mnist download 

In [34]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
np.random.seed(42)

# 훈련 및 테스트 데이터 로딩
(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.mnist.load_data()
cv2_imshow(x_train[0]) #Colab opencv imshow 
y_train[0]
num_classes = 10 # 클래스는 0부터 9까지의 숫자

# 이미지를 칼럼 벡터(NN의 입력으로)로 변환 
x_train, x_test = x_train.reshape(-1, 28*28), x_test.reshape(-1, 28*28)
# 레이블을 원-핫 벡터(NN의 결과)로 변환(예를 들어 레이블 4를 벡터 '[0,0,0,0,1,0,0,0,0,0]' 로 변환)
y_train = np.eye(num_classes)[y_train]
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')

x_train.shape : (60000, 784)
y_train.shape : (60000, 10)


> Class FullyConnectedLayer

In [35]:
class FullyConnectedLayer(object) :
  """ A simple fully-connected NN layer.
  Args :
    num_imputs (int) : input vector size / input data count
    layer_size (int) : output vector size / neurun count
    activation_fn (callable) : 이 계층에 사용될 활성화 함수  
  Attributes : 
    W (ndarray) : 입력값에 대한 가중치
    b (ndarray) : 가중합에 더해질 편향값 
    size (int) : 계층의 크기 / 뉴런의 개수 
    activation_fn(callable):뉴런에 적용할 활성화 함수
  """

  def __init__(self, num_inputs, layer_size, activation_fn) :
    super().__init__()
    #임의로 매개변수를 초기화(여기서는 정규 분포를 사용)
    self.W = np.random.standard_normal((num_inputs, layer_size))
    self.b = np.random.standard_normal(layer_size)
    self.size = layer_size
    self.activation_fn = activation_fn

  def forward(self, x):
    """ 계층을 통해 입력 신호를 전달"""
    z = np.dot(x, self.W) + self.b
    return self.activation_fn(z)

> 네트워크 구현하기 


In [36]:
import numpy as np

def sigmoid (x): # x의 요소에 시그모이드 함수를 적용
  return 1 / (1 + np.exp(-x)) #y 

class SimpleNetwork(object) :
  """ 간단한 완전 연결 신경망
  Args :
    num_inputs (int) : 입력 벡터 크기 / 입력 값 개수
    num_outputs(int) : 출력 벡터 크기
    hidden_layers_sizes(list) : 네트워크에 추가될 은닉 계층의 크기를 포함한 리스트 
  Attributes : 
    layers (list) : 이 간단한 네트워크를 구성하는 계층 리스트 
  """

  def __init__(self, num_inputs, num_outputs, hidden_layers_sizes=(64,32)):
    super().__init__()
    #네트워크를 구성하는 계층 리스트를 구성
    sizes = [num_inputs, *hidden_layers_sizes, num_outputs]
    print(sizes)
    self.layers = [
                   FullyConnectedLayer(sizes[i], sizes[i+1], sigmoid)
                   for i in range(len(sizes)-1) ]    
    print(self.layers)

  def forward(self, x):
    """입력 벡터 'x'를 계층을 통해 전달 """
    for layer in self.layers : # from the input layer to the output one
      x = layer.forward(x)
      #print(x)
      
    return x 

  def predict(self, x):
    """ 'x'에 대응하는 출력을 계산하고 출력값이 가장 큰 인덱스를 반환 """
    estimations = self.forward(x)
    best_class = np.argmax(estimations)

    return best_class

  def evaluate_accuracy(self, x_val, y_val) :
    """ 검증 데이터셋을 사용해 네트워크의 정확도를 평가 """
    num_corrects = 0
    for i in range(len(x_val)) :
      if self.predict(x_val[i]) == y_val[i] :
        num_corrects += 1
    return num_corrects / len(x_val)

> Main

In [37]:
# MNIST images 분류를 위한 네트워크, 그 안에 크기가 64, 32인 2개의 은닉 계층이 있음 
# SimpleNetwork(784, 10, [64x,32])
mnist_classifier = SimpleNetwork(x_train.shape[1], num_classes, [64,32])
# ... 그리고 MNIST 데이터셋에서 이 네트워크의 정확도를 평가함 
accuracy  = mnist_classifier.evaluate_accuracy(x_test, y_test)
print("accuracy = {:.2f}%".format(accuracy * 100))

[784, 64, 32, 10]
[<__main__.FullyConnectedLayer object at 0x7f43f7d3e610>, <__main__.FullyConnectedLayer object at 0x7f43f7c29450>, <__main__.FullyConnectedLayer object at 0x7f43f7c29d50>]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


accuracy = 10.95%
